In [54]:
# Initial imports 
import pandas as pd
from pathlib import Path
from sklearn import tree
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

In [57]:
#read in csv file and convert to padas DF
file_path = Path("Starting_Data/cleaned_data_revisedSet_noNull2.csv")
df = pd.read_csv(file_path)
df.head()


,max_ma,min_ma,genus,lng,lat,lithology1,environment,time_bins,Orn0Saur1
0,83.5,70.6,Centrosaurus,-111.528931,50.737297,not reported,terrestrial,Campanian,0
1,83.5,70.6,Gryposaurus,-111.529999,50.770000,not reported,terrestrial,Campanian,0
2,70.6,66.0,Ankylosaurus,-106.569901,47.637699,not reported,terrestrial,Maastrichtian,0
3,72.1,66.0,Pachycephalosaurus,-106.227776,47.695831,sandstone,channel,Maastrichtian,0
4,157.3,145.0,Stegosaurus,-106.389999,42.035831,not reported,terrestrial,Tithonian,0


In [83]:
# Encode the genus column
dummies_df = pd.get_dummies(\
    data = df, columns= [
        'lithology1', 
        'environment'
        ]
    )\
    .drop(
        ['genus',
            'Orn0Saur1',
         'lat',
         'time_bins'
         ], 
         axis=1
    )
dummies_df.head()

,max_ma,min_ma,lng,lithology1_amber,lithology1_bindstone,lithology1_breccia,lithology1_carbonate,lithology1_chalk,lithology1_chert,lithology1_claystone,...,environment_pond,environment_prodelta,environment_sand shoal,environment_shallow subtidal,environment_shoreface,environment_sinkhole,environment_spring,environment_terrestrial,environment_transition zone,environment_wet floodplain
0,83.5,70.6,-111.528931,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,83.5,70.6,-111.529999,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,70.6,66.0,-106.569901,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,72.1,66.0,-106.227776,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,157.3,145.0,-106.389999,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [84]:
df.columns

Index(['max_ma', 'min_ma', 'genus', 'lng', 'lat', 'lithology1', 'environment',
       'time_bins', 'Orn0Saur1'],
      dtype='object')

In [85]:
# Scale data using StandardScaler and fit_transform
scaled_features = StandardScaler().\
    fit_transform(
        df[[
            'max_ma', 
            'min_ma',
            'lat'
                ]]
        )

# Diplay the first five rows of the scaled data
scaled_features[0:5]

array([[-0.8058735 , -0.88505555,  0.6856713 ],
       [-0.8058735 , -0.88505555,  0.68710525],
       [-1.05827742, -0.97647526,  0.54976137],
       [-1.02892813, -0.97647526,  0.55231032],
       [ 0.63811168,  0.59355892,  0.30413291]])

In [86]:
# create a dataframe with dummies and scaled data
scaled_df = pd.DataFrame(
    scaled_features,
    columns= [
            'max_ma', 
            'min_ma'
                ]
)
# review results
scaled_df.head()

ValueError: Shape of passed values is (8313, 3), indices imply (8313, 2)

In [87]:
# concatenate genus_dummies to scaled df
scaled_df = pd.concat(
    [
        scaled_df,
        dummies_df
    ], 
    axis=1
)

# review results
scaled_df.head()


,max_ma,min_ma,max_ma,min_ma,lithology1_amber,lithology1_bindstone,lithology1_breccia,lithology1_carbonate,lithology1_chalk,lithology1_chert,...,environment_pond,environment_prodelta,environment_sand shoal,environment_shallow subtidal,environment_shoreface,environment_sinkhole,environment_spring,environment_terrestrial,environment_transition zone,environment_wet floodplain
0,-0.805874,-0.885056,83.5,70.6,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,-0.805874,-0.885056,83.5,70.6,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,-1.058277,-0.976475,70.6,66.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,-1.028928,-0.976475,72.1,66.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0.638112,0.593559,157.3,145.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [88]:
# Define features set
X = scaled_df.copy()

# Define target vector
y = df["Orn0Saur1"].values.reshape(-1, 1)
y[:5]

array([[0],
       [0],
       [0],
       [0],
       [0]], dtype=int64)

In [89]:
# Splitting into Train and Test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)


In [90]:
# Creating StandardScaler instance
scaler = StandardScaler()
# Fitting Standard Scaller
X_scaler = scaler.fit(X_train)
# Scaling data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


In [91]:
# Creating the decision tree classifier instance
model = tree.DecisionTreeClassifier()
# Fitting the model
model = model.fit(X_train_scaled, y_train)

In [92]:
# Making predictions using the testing data
predictions = model.predict(X_test_scaled)

In [93]:
# Calculating the confusion matrix
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)

# Calculating the accuracy score
acc_score = accuracy_score(y_test, predictions)



In [94]:
# Displaying results
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))


Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,378,294
Actual 1,258,1149


Accuracy Score : 0.7344877344877345
Classification Report
              precision    recall  f1-score   support

           0       0.59      0.56      0.58       672
           1       0.80      0.82      0.81      1407

    accuracy                           0.73      2079
   macro avg       0.70      0.69      0.69      2079
weighted avg       0.73      0.73      0.73      2079



# Conclusion
These results are phenomenal. I ran this model without our geodata because it seems we need another approach when incorporating geodata\
This model is clearly efficient, though it tends to overestimate specimens as therapods. I recommend running this model again with a more balanced data set.a